# For 100x100m cells inside each 1 km buffer, many of those will not be liquefaction. Let's identify those and drop them.

In [1]:
# Alex Chansky
# 3/14/21
# Purpose: Identifying and dropping non-liq cells near liq cells
import arcpy
from arcpy import env
import pandas as pd
import os
arcpy.CheckOutExtension("Spatial") # This checks out the license
from arcpy.sa import *
# If getting "Not Signed into Portal" message, should open up and log into ArcGIS Pro

In [26]:
arcpy.env.overwriteOutput = True

In [10]:
import datetime
region_path = "C:\\Users\\achans01\\Downloads\\WatBods\\"
path = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\"
temp_path = "C:\\Users\\achans01\\Downloads\\Arc"
eq_list = pd.read_csv("R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\z_eq_names\\2021_eq_names.csv")
num_eq = eq_list.shape[0]
eq_list_df = pd.DataFrame(eq_list)

In [18]:
def ptsonly(eq_name):
    points = path + eq_name + "\\Liq_point.shp"
    fishnet = path + eq_name + "\\fish_liq_poly.shp"
    fish_feature_count = path + eq_name + "\\fish_liq_poly_count.shp"
    
    # Type KEEP_COMMON only keeps cells which overlap points/polys
    arcpy.analysis.SpatialJoin(fishnet, points, fish_feature_count,"","KEEP_COMMON")

In [19]:
def polyonly(eq_name):
    poly = path + eq_name + "\\Liq_poly.shp"
    fishnet = path + eq_name + "\\fish_liq_poly.shp"
    fish_feature_count = path + eq_name + "\\fish_liq_poly_count.shp"
    
    # Type KEEP_COMMON only keeps cells which overlap points/polys
    arcpy.analysis.SpatialJoin(fishnet, poly, fish_feature_count,"","KEEP_COMMON")

In [23]:
def both(eq_name):
    fishnet = path + eq_name + "\\fish_liq_poly.shp"
    
    points = path + eq_name + "\\Liq_point.shp"
    fish_feature_count_ptonly = path + eq_name + "\\fish_liq_poly_count_ptonly.shp"
    # Type KEEP_COMMON only keeps cells which overlap points/polys
    arcpy.analysis.SpatialJoin(fishnet, points, fish_feature_count_ptonly,"","KEEP_COMMON")
    
    poly = path + eq_name + "\\Liq_poly.shp"
    fish_feature_count_polonly = path + eq_name + "\\fish_liq_poly_count_polonly.shp"
    # Type KEEP_COMMON only keeps cells which overlap points/polys
    arcpy.analysis.SpatialJoin(fishnet, poly, fish_feature_count_polonly,"","KEEP_COMMON")
    
    
    fish_feature_count = path + eq_name + "\\fish_liq_poly_count.shp"
    arcpy.management.Merge([fish_feature_count_ptonly, fish_feature_count_polonly], fish_feature_count, "Sum")
    # Sum does NOT work properly, but all of the correct cells are saved

In [24]:
# Join_Count is automatically included, so output of spatial join will have number of points or
# polygons within a cell

In [28]:
for x in range(10,num_eq):
    ct = datetime.datetime.now() 
    print("current time:-", ct)
    
    eq_name = eq_list['NewFolder'][x]
    if eq_list['Geometry'][x] == "PtsOnly" and eq_list['Working'][x]==1:
        ptsonly(eq_name)
    elif eq_list['Geometry'][x] == "PolyOnly" and eq_list['Working'][x]==1:
        polyonly(eq_name)
    elif eq_list['Geometry'][x] == "Both" and eq_list['Working'][x]==1:
        both(eq_name)
    else:
        pass
    
    print("Successfully dropped cells for " + str(x), eq_name)
    ct = datetime.datetime.now() 
    print("current time:-", ct)

current time:- 2021-03-14 22:38:55.928722
Successfully dropped cells for 10 Christchurch
current time:- 2021-03-14 22:39:26.478961
current time:- 2021-03-14 22:39:26.478961
Successfully dropped cells for 11 Darfield
current time:- 2021-03-14 22:40:05.154943
current time:- 2021-03-14 22:40:05.154943
Successfully dropped cells for 12 Denali
current time:- 2021-03-14 22:40:10.318206
current time:- 2021-03-14 22:40:10.318206
Successfully dropped cells for 13 Duzce
current time:- 2021-03-14 22:40:14.625286
current time:- 2021-03-14 22:40:14.625286
Successfully dropped cells for 14 Emilia
current time:- 2021-03-14 22:40:20.065882
current time:- 2021-03-14 22:40:20.065882
Successfully dropped cells for 15 Haiti
current time:- 2021-03-14 22:40:25.305389
current time:- 2021-03-14 22:40:25.305389
Successfully dropped cells for 16 HectorMine
current time:- 2021-03-14 22:40:25.305389
current time:- 2021-03-14 22:40:25.305389
Successfully dropped cells for 17 Hokkaido
current time:- 2021-03-14 22:4

In [ ]:
# Select all where Count = 0 and delete